In [4]:
!pip install sagemaker-experiments


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import sagemaker
import boto3
import pandas as pd
from datetime import datetime 

from sagemaker import image_uris 
from sagemaker.session import Session
from sagemaker.inputs import TrainingInput
from time import gmtime, strftime
import time
from smexperiments import experiment

region = boto3.Session().region_name

session = sagemaker.Session()
bucket = 'lawsnic-aiml-east2'
prefix = 'kaggle/customerChurn'

role = sagemaker.get_execution_role()

sm = boto3.Session().client(service_name='sagemaker',region_name=region)


# name of experiment
timestep = datetime.now()
timestep = timestep.strftime("%d-%m-%Y-%H-%M-%S")
experiment_name = timestep + "xgboost-cust-churn"

# create experiment
experiment.Experiment.create(
    experiment_name=experiment_name,
    description="Iterative model tuning - altering data engineering steps",
    sagemaker_boto_client=sm,
)

Experiment(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f30db3e5410>,experiment_name='18-02-2023-18-48-10xgboost-cust-churn',description='Iterative model tuning - altering data engineering steps',tags=None,experiment_arn='arn:aws:sagemaker:us-east-2:791580863750:experiment/18-02-2023-18-48-10xgboost-cust-churn',response_metadata={'RequestId': '2ac3036e-d866-4106-a727-61aed252e5db', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '2ac3036e-d866-4106-a727-61aed252e5db', 'content-type': 'application/x-amz-json-1.1', 'content-length': '109', 'date': 'Sat, 18 Feb 2023 18:48:09 GMT'}, 'RetryAttempts': 0})

In [6]:
#sm.get_paginator('list_candidates_for_auto_ml_job')
#sm.list_candidates_for_auto_ml_job(AutoMLJobName='xgb-custChurn-2023-2-18-11')

In [7]:
# cell 10
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [8]:
# cell 11
#validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T17-24-14/part-00000-3986a971-e8a5-4844-bfc8-e72e78f341b4-c000.csv'
#training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T17-24-14/part-00000-0338ba51-05d0-4737-bd40-d52e24739eb6-c000.csv'

training_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train/CustomerChurnDW-2023-01-06T14-14-13/part-00000-642a0746-49b9-4fda-ad2d-98afe1db11ec-c000.csv'
validation_location = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/test/CustomerChurnDW-2023-01-06T14-14-13/part-00000-ccd6e4dd-898c-4fc4-a63a-85d1cfcfc4dc-c000.csv'

In [98]:
train_data = pd.read_csv(training_location)
train_data["SeniorCitizen"] = train_data["SeniorCitizen"].astype(int)
train_data["Churn"] = train_data["Churn"].astype(int)
train_data["Partner"] = train_data["Partner"].astype(int)
train_data["Dependents"] = train_data["Dependents"].astype(int)
train_data["PhoneService"] = train_data["PhoneService"].astype(int)
train_data["PaperlessBilling"] = train_data["PaperlessBilling"].astype(int)
train_data["isFemale"] = train_data["isFemale"].astype(int) 
train_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv',index=False)
train_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv',header=None,index=False)

print('train data has {} features'.format(len(train_data.columns)))

test_data = pd.read_csv(validation_location)
test_data["SeniorCitizen"] = test_data["SeniorCitizen"].astype(int)
test_data["Churn"] = test_data["Churn"].astype(int)
test_data["Partner"] = test_data["Partner"].astype(int)
test_data["Dependents"] = test_data["Dependents"].astype(int)
test_data["PhoneService"] = test_data["PhoneService"].astype(int)
test_data["PaperlessBilling"] = test_data["PaperlessBilling"].astype(int)
test_data["isFemale"] = test_data["isFemale"].astype(int)
test_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target_and_header.csv',index=False)
test_data.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target.csv',header=None,index=False)

print('validation data has {} features before dropping target'.format(len(test_data.columns)))

test_data2 =test_data.drop(["Churn"], axis=1) 
test_data2.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv',header=None,index=False)
test_data2.to_csv('s3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_header.csv',index=False)


s3_uri_train_data = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train.csv'
s3_uri_train_data_with_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv'
s3_uri_validate_data = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate.csv'
s3_uri_validate_data_with_target_and_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target_and_header.csv'
s3_uri_validate_data_with_target = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_target.csv'
s3_uri_validate_data_with_header = 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/validate_with_header.csv'

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=s3_uri_train_data, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=s3_uri_validate_data_with_target, content_type='csv')

train data has 41 features
validation data has 41 features before dropping target


First we'll need to specify training parameters to the estimator.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [10]:
from smexperiments.trial import Trial

trial = Trial.create(experiment_name=experiment_name, sagemaker_boto_client=sm)
experiment_config = {
    "ExperimentName": experiment_name,
    "TrialName": trial.trial_name,
    "TrialComponentDisplayName": "Data",
}

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/manual-output/xgb-hpo'.format(bucket,prefix),
                                    sagemaker_session=sess,
                                    enable_sagemaker_metrics=True)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation},
    experiment_config=experiment_config)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: xgboost-2023-02-18-18-48-12-247


2023-02-18 18:48:12 Starting - Starting the training job...
2023-02-18 18:48:35 Starting - Preparing the instances for trainingProfilerReport-1676746092: InProgress
......
2023-02-18 18:49:35 Downloading - Downloading input data...
2023-02-18 18:49:56 Training - Downloading the training image......
2023-02-18 18:51:07 Uploading - Uploading generated training modelArguments: train
[2023-02-18:18:50:58:INFO] Running standalone xgboost training.
[2023-02-18:18:50:58:INFO] File size need to be processed in the node: 1.03mb. Available memory size in the node: 8617.6mb
[2023-02-18:18:50:58:INFO] Determined delimiter of CSV input is ','
[18:50:58] S3DistributionType set as FullyReplicated
[18:50:58] 6337x40 matrix with 253480 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2023-02-18:18:50:58:INFO] Determined delimiter of CSV input is ','
[18:50:58] S3DistributionType set as FullyReplicated
[18:50:58] 706x40 matrix with 28240 entries loaded from /opt/ml/inp

## HPO

In [11]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)

In [12]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating hyperparameter tuning job with name: xgboost-230218-1851


............................................................................................!


In [13]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [14]:
boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())['FinalMetricDataList']

[{'MetricName': 'validation:auc',
  'Value': 0.819957971572876,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 59, 9, tzinfo=tzlocal())},
 {'MetricName': 'train:auc',
  'Value': 0.796550989151001,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 59, 9, tzinfo=tzlocal())},
 {'MetricName': 'ObjectiveMetric',
  'Value': 0.819957971572876,
  'Timestamp': datetime.datetime(2023, 2, 18, 18, 59, 9, tzinfo=tzlocal())}]

In [32]:
#modeltuner.best_training_job()
#boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())

s3_uri_model= boto3.client('sagemaker').describe_training_job(TrainingJobName=tuner.best_training_job())['ModelArtifacts']['S3ModelArtifacts']
print(s3_uri_model)

s3://lawsnic-aiml-east2/kaggle/customerChurn/manual-output/xgb-hpo/xgboost-230218-1851-019-a6ee6d9a/output/model.tar.gz


In [30]:
#xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')
#predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

model_name = 'xgbHPO-custChurn-model-' + str(int(time.time()))

In [35]:
primary_container = {"Image": container, "ModelDataUrl": s3_uri_model}

model = sm.create_model( 
    PrimaryContainer=primary_container,
    ModelName=model_name,
    ExecutionRoleArn=role
)

## Batch Inference

In [38]:
from time import gmtime, strftime, sleep 
from sagemaker.inputs import BatchDataCaptureConfig
from sagemaker.model_monitor import DataCaptureConfig
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

transform_job_name = 'xgbHPO-custchurn-batch-' + timestamp_suffix

transform_input = {
        'DataSource': {
            'S3DataSource': {
                'S3DataType': 'S3Prefix',
                'S3Uri': s3_uri_validate_data_with_header
                #'S3Uri': s3_uri_validate_data
            }
        },
        'ContentType': 'text/csv',
        'CompressionType': 'None',
        'SplitType': 'Line'
    }

transform_output = {
        'S3OutputPath': 's3://{}/{}/inference-results/xgbHPO/'.format(bucket, prefix),
        "Accept": "text/csv",
        'AssembleWith': 'Line',
    }

transform_resources = {
        'InstanceType': 'ml.m5.4xlarge',
        'InstanceCount': 1
    }

data_capture_config = {
       "DestinationS3Uri": 's3://{}/{}/model-monitor/batch-results/xgbHPO'.format(bucket,prefix), 
       "GenerateInferenceId": True,
    }

data_proc = { 
        "JoinSource": "Input"
        }
    
transformer = sm.create_transform_job(TransformJobName = transform_job_name,
                        ModelName = model_name,
                        TransformInput = transform_input,
                        TransformOutput = transform_output,
                        TransformResources = transform_resources,
                        DataCaptureConfig= data_capture_config,
                        DataProcessing = data_proc
)

In [39]:
print ('JobStatus')
print('----------')


describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
job_run_status = describe_response['TransformJobStatus']
print (job_run_status)

while job_run_status not in ('Failed', 'Completed', 'Stopped'):
    describe_response = sm.describe_transform_job(TransformJobName = transform_job_name)
    job_run_status = describe_response['TransformJobStatus']
    print (job_run_status)
    sleep(30)

JobStatus
----------
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
InProgress
Completed


In [42]:
s3_output_key = '{}/inference-results/xgbHPO/validate.csv.out'.format(prefix);
local_inference_results_path = 'inference_results.csv'

s3 = boto3.resource('s3')
inference_results_bucket = s3.Bucket(bucket)

print(s3_output_key)

kaggle/customerChurn/inference-results/xgbHPO/validate.csv.out


In [44]:
inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

data = pd.read_csv(local_inference_results_path, sep=',')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
data

,0,0.1,0.2,10,0.3,0.4,29.75,301.9,1,0.0,...,0.0.16,1.0.8,0.0.17,0.0.18,1.0.9,0.0.19,0.0.20,0.08055959641933441,f830f3d7-d9fc-4014-ab56-074ca136f913,2023-02-18T19:31:10Z
0,0,0,0,16,1,0,18.95,326.80,0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.033425,6d8e4a26-4e92-42f0-8894-c8c78ae71679,2023-02-18T19:31:10Z
1,0,1,1,10,1,0,55.20,528.35,1,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.017306,6e21a5ee-2e75-4398-98b3-17a55c8d1dbc,2023-02-18T19:31:10Z
2,0,0,1,17,1,1,64.70,1093.10,1,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.079891,e9c4fac4-fe1f-4273-8970-eeb75cf4251c,2023-02-18T19:31:10Z
3,1,1,0,71,1,1,96.35,6766.95,1,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.043323,284e2e62-8f4b-45b7-be7d-df115866da51,2023-02-18T19:31:10Z
4,1,1,1,60,1,1,74.85,4456.35,1,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.023446,b3c731a4-7cf4-4a49-ae28-e41ec4fff905,2023-02-18T19:31:10Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
700,0,0,0,26,0,0,35.75,1022.50,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.120930,56fd25f1-2af0-494f-ba7b-3fa2dec75a67,2023-02-18T19:31:10Z
701,0,0,0,38,1,1,95.10,3691.20,0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.369698,824730cb-3166-4de7-85a2-14876375d962,2023-02-18T19:31:10Z
702,0,0,0,51,1,0,20.65,1020.75,1,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.033425,1c5bf318-63d2-4f65-95d9-f546145a6002,2023-02-18T19:31:10Z
703,0,1,1,1,1,1,70.65,70.65,0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.045573,b3589879-f447-4a5a-940e-c2f07d779d5d,2023-02-18T19:31:10Z


In [45]:
s3_client = boto3.Session().client('s3') 
#result = s3_client.list_objects('s3://{}/{}/model-monitor/batch-results'.format(bucket,prefix))
current_endpoint_capture_prefix = '{}/{}'.format(prefix, 'model-monitor/batch-results/xgbHPO')
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
kaggle/customerChurn/model-monitor/batch-results/xgbHPO/input/2023/02/18/19/373a764c-46c9-4daa-a259-1808d6f57135.json
 kaggle/customerChurn/model-monitor/batch-results/xgbHPO/output/2023/02/18/19/c1e13ffb-c23c-49d8-9f0d-881acd702f79.json


In [46]:
baseline_prefix = prefix + '/baselining/xgbHPO'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://lawsnic-aiml-east2/kaggle/customerChurn/baselining/xgbHPO/data
Baseline results uri: s3://lawsnic-aiml-east2/kaggle/customerChurn/baselining/xgbHPO/results


### Create Baseline job

In [50]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor_baseline = my_default_monitor.suggest_baseline(
    baseline_dataset=s3_uri_train_data_with_header,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True,
    logs = False
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2023-02-18-19-45-03-261



Job Name:  baseline-suggestion-job-2023-02-18-19-45-03-261
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://lawsnic-aiml-east2/kaggle/customerChurn/features/partial/train_with_header.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://lawsnic-aiml-east2/kaggle/customerChurn/baselining/xgbHPO/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
....................................................................!

### Explore Generated Model Quality Data

In [51]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
kaggle/customerChurn/baselining/xgbHPO/results/constraints.json
 kaggle/customerChurn/baselining/xgbHPO/results/statistics.json


In [86]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,SeniorCitizen,Integral,6337,0,0.165536,1049.00,0.371663,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
1,Partner,Integral,6337,0,0.484930,3073.00,0.499773,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0,..."
2,Dependents,Integral,6337,0,0.299195,1896.00,0.457905,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,tenure,Integral,6337,0,32.410289,205384.00,24.591410,0.00,72.00,"[{'lower_bound': 0.0, 'upper_bound': 7.2, 'cou...",0.64,2048.0,"[[48.0, 72.0, 7.0, 38.0, 10.0, 30.0, 31.0, 46...."
4,PhoneService,Integral,6337,0,0.903267,5724.00,0.295594,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
5,PaperlessBilling,Integral,6337,0,0.593656,3762.00,0.491150,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0,..."
6,MonthlyCharges,Fractional,6337,0,64.865575,411053.15,30.103892,18.25,118.75,"[{'lower_bound': 18.25, 'upper_bound': 28.3, '...",0.64,2048.0,"[[64.4, 84.9, 19.3, 83.9, 104.4, 74.65, 59.05,..."
7,TotalCharges,Fractional,6327,10,2290.958527,14494894.60,2273.146587,18.80,8684.80,"[{'lower_bound': 18.8, 'upper_bound': 885.4, '...",0.64,2048.0,"[[3255.35, 1067.65, 2065.4, 4136.4, 221.9, 384..."
8,Churn,Integral,6337,0,0.267003,1692.00,0.442394,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
9,isFemale,Integral,6337,0,0.497081,3150.00,0.499991,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0,..."


In [53]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,SeniorCitizen,Integral,1.000000,True
1,Partner,Integral,1.000000,True
2,Dependents,Integral,1.000000,True
3,tenure,Integral,1.000000,True
4,PhoneService,Integral,1.000000,True
5,PaperlessBilling,Integral,1.000000,True
6,MonthlyCharges,Fractional,1.000000,True
7,TotalCharges,Fractional,0.998422,True
8,Churn,Integral,1.000000,True
9,isFemale,Integral,1.000000,True


### Set up Model Quality

In [118]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

In [64]:
model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session
)

In [104]:
inference_results_bucket.download_file(s3_output_key, local_inference_results_path);

#inference_results_pd = pd.read_csv(local_inference_results_path, sep=',', header=None)
inference_results_pd = pd.read_csv(local_inference_results_path, sep=',')
pd.set_option('display.max_rows', 10)         # Keep the output on one page
inference_results_pd.head()

,0,0.1,0.2,10,0.3,0.4,29.75,301.9,1,0.0,...,0.0.16,1.0.8,0.0.17,0.0.18,1.0.9,0.0.19,0.0.20,0.08055959641933441,f830f3d7-d9fc-4014-ab56-074ca136f913,2023-02-18T19:31:10Z
0,0,0,0,16,1,0,18.95,326.80,0,1.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.033425,6d8e4a26-4e92-42f0-8894-c8c78ae71679,2023-02-18T19:31:10Z
1,0,1,1,10,1,0,55.20,528.35,1,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.017306,6e21a5ee-2e75-4398-98b3-17a55c8d1dbc,2023-02-18T19:31:10Z
2,0,0,1,17,1,1,64.70,1093.10,1,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.079891,e9c4fac4-fe1f-4273-8970-eeb75cf4251c,2023-02-18T19:31:10Z
3,1,1,0,71,1,1,96.35,6766.95,1,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.043323,284e2e62-8f4b-45b7-be7d-df115866da51,2023-02-18T19:31:10Z
4,1,1,1,60,1,1,74.85,4456.35,1,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.023446,b3c731a4-7cf4-4a49-ae28-e41ec4fff905,2023-02-18T19:31:10Z


In [106]:
temp = pd.Index(['prediction']).append(test_data2.columns).append(pd.Index(['inference id', 'inference timestamp']))

inference_results_pd.columns = temp 

In [107]:
inference_results_pd.insert(loc=0, column='Churn', value = test_data['Churn'])

inference_with_groundtruth_data_uri = 's3://lawsnic-aiml-east2/kaggle/customerChurn/model-monitor/xgbHPO/model-quality/inference_with_groundtruth.csv'

inference_results_pd.to_csv(inference_with_groundtruth_data_uri, index=False)


inference_results_pd.head()

,Churn,prediction,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,...,OnlineBackup_Yes,OnlineBackup_No internet service,DeviceProtection_No,DeviceProtection_Yes,DeviceProtection_No internet service,Contract_Month-to-month,Contract_Two year,Contract_One year,inference id,inference timestamp
0,0,0,0,0,16,1,0,18.95,326.80,0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.033425,6d8e4a26-4e92-42f0-8894-c8c78ae71679,2023-02-18T19:31:10Z
1,0,0,1,1,10,1,0,55.20,528.35,1,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.017306,6e21a5ee-2e75-4398-98b3-17a55c8d1dbc,2023-02-18T19:31:10Z
2,1,0,0,1,17,1,1,64.70,1093.10,1,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.079891,e9c4fac4-fe1f-4273-8970-eeb75cf4251c,2023-02-18T19:31:10Z
3,1,1,1,0,71,1,1,96.35,6766.95,1,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.043323,284e2e62-8f4b-45b7-be7d-df115866da51,2023-02-18T19:31:10Z
4,0,1,1,1,60,1,1,74.85,4456.35,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.023446,b3c731a4-7cf4-4a49-ae28-e41ec4fff905,2023-02-18T19:31:10Z


In [108]:
inference_with_groundtruth_data_uri = 's3://lawsnic-aiml-east2/kaggle/customerChurn/model-monitor/model-quality/xgb_inference_with_groundtruth.csv'
inference_results_pd.to_csv(inference_with_groundtruth_data_uri, index=False)

In [110]:
baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)

baseline_job_name = "XGB-CustChurn-ModelQualityBaseline1"
job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    #baseline_dataset=baseline_data_uri, # The S3 location of the validation dataset.
    baseline_dataset=inference_with_groundtruth_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri = baseline_results_uri, # The S3 location to store the results.
    problem_type='BinaryClassification',
    inference_attribute= "prediction", # The column in the dataset that contains predictions.
    #probability_attribute= "probability", # The column in the dataset that contains probabilities.
    ground_truth_attribute= "Churn" # The column in the dataset that contains ground truth labels.
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name XGB-CustChurn-ModelQualityBaseline1



Job Name:  XGB-CustChurn-ModelQualityBaseline1
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://lawsnic-aiml-east2/kaggle/customerChurn/model-monitor/model-quality/xgb_inference_with_groundtruth.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://lawsnic-aiml-east2/kaggle/customerChurn/baselining/xgbHPO/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..................................................................!

In [111]:
baseline_job = model_quality_monitor.latest_baselining_job

In [112]:
import pandas as pd
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T

,threshold,comparison_operator
recall,0.158192,LessThanThreshold
precision,0.301075,LessThanThreshold
accuracy,0.696454,LessThanThreshold
true_positive_rate,0.158192,LessThanThreshold
true_negative_rate,0.876894,LessThanThreshold
false_positive_rate,0.123106,GreaterThanThreshold
false_negative_rate,0.841808,GreaterThanThreshold
f0_5,0.255009,LessThanThreshold
f1,0.207407,LessThanThreshold
f2,0.174782,LessThanThreshold


In [113]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get("Contents")]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
kaggle/customerChurn/baselining/xgbHPO/results/constraints.json
 kaggle/customerChurn/baselining/xgbHPO/results/statistics.json


In [114]:
baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(
    baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
)
schema_df.transpose().head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,0
confusion_matrix.0.0,463.000000
confusion_matrix.0.1,65.000000
confusion_matrix.1.0,149.000000
confusion_matrix.1.1,28.000000
recall.value,0.158192
recall.standard_deviation,0.012988
precision.value,0.301075
precision.standard_deviation,0.011265
accuracy.value,0.696454
accuracy.standard_deviation,0.007815


In [115]:
constraints_df = pd.io.json.json_normalize(
    baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]
)
constraints_df.transpose().head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


,0
recall.threshold,0.158192
recall.comparison_operator,LessThanThreshold
precision.threshold,0.301075
precision.comparison_operator,LessThanThreshold
accuracy.threshold,0.696454
accuracy.comparison_operator,LessThanThreshold
true_positive_rate.threshold,0.158192
true_positive_rate.comparison_operator,LessThanThreshold
true_negative_rate.threshold,0.876894
true_negative_rate.comparison_operator,LessThanThreshold


In [116]:
statistics_path = "{}/statistics.json".format(baseline_results_uri)
constraints_path = "{}/constraints.json".format(baseline_results_uri)

### Configure Modle Quality checks

There are two configurations we create here, one is CheckJobConfig and the other is ModelQualityCheckConfig. The CheckJobConfig is used to configure the underlying processing job used by Model Monitor. This is where users can specify the role, instance type, etc.

The ModelQualityCheckConfig is used to configure how Model Monitor runs the model quality check. It accepts an argument called baseline_dataset. This is the dataset that is passed to the transform job. The dataset passed here is compared against the baseline and statistics file generated by the suggest_baseline method.

AttributeError: 'dict' object has no attribute 'output_path'